In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import split
from pyspark.sql.functions import sum
from pyspark.sql.functions import coalesce
#from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_format

Crear una instancia de SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/24 11:47:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/24 11:47:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
aviation = pd.read_csv("AccidentesAviones.csv")#, header = True)

In [4]:
aviation.head(3)

,id,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Unnamed: 18
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",NaN
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,NaN
2,2,"July 12, 1912",630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,NaN


Leer el archivo CSV como un DataFrame de PySpark

In [5]:
aviation = spark.createDataFrame(aviation)

In [6]:
from pyspark.sql.functions import year, to_date
from pyspark.sql.types import IntegerType

aviation = aviation.withColumn("date", to_date(aviation.fecha, "MMMM dd, yyyy"))
aviation = aviation.withColumn("year", year(aviation.date).cast(IntegerType()))


In [7]:
aviation.printSchema

<bound method DataFrame.printSchema of DataFrame[id: bigint, fecha: string, HORA declarada: string, Ruta: string, OperadOR: string, flight_no: string, route: string, ac_type: string, registration: string, cn_ln: string, all_aboard: string, PASAJEROS A BORDO: string, crew_aboard: string, cantidad de fallecidos: string, passenger_fatalities: string, crew_fatalities: string, ground: string, summary: string, Unnamed: 18: string, date: date, year: int]>

In [8]:
aviation = aviation.drop("id","fecha","HORA declarada","summary","Unnamed: 18","date")

En el contexto de la aviación, "cn_ln" generalmente se refiere al número de construcción o número de serie de una aeronave. Cada aeronave fabricada por un fabricante determinado tiene un número de serie único que se utiliza para identificarla de manera única. El número de construcción puede ser asignado por el fabricante de la aeronave y se utiliza ampliamente en la industria de la aviación para rastrear y gestionar el historial de una aeronave específica.

In [9]:
aviation = aviation.drop("cn_ln")

23/05/24 11:47:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


En la aeronáutica, "ac_type" hace referencia al tipo de aeronave. Es común utilizar esta columna para indicar el modelo o tipo de aeronave involucrada en un evento o registro de aviación. 

In [10]:
aviation = aviation.drop("ac_type")

In [11]:
aviation = aviation.drop("registration")

In [12]:
aviation = aviation.drop("OperadOR")

In [13]:
aviation = aviation.drop("flight_no")

In [14]:
aviation = aviation.withColumnRenamed("PASAJEROS A BORDO", "passengers")\
                   .withColumnRenamed("crew_aboard", "crew")\
                   .withColumnRenamed("cantidad de fallecidos", "fatalities")\
                   .withColumnRenamed("Ruta", "location") 

In [15]:
aviation = aviation.filter(aviation.year >= 1970)

In [16]:
aviation.limit(3).pandas_api()

/usr/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


,location,route,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,year
0,"Stockholm, Sweden",Stockholm - Zurich,10,7,3,5,5,0,0,1970
1,Near Villia Greece,Military exercise,27,22,5,23,19,4,0,1970
2,"Faleolo, Western Samoa","Apia, Western Samoa - Pago Pago, American Samoa",32,29,3,32,29,3,0,1970


In [17]:
# Calcular el total de fatalidades y pasajeros por año en un solo paso
aviaton_year = aviation.groupBy("year").agg(sum("all_aboard").alias("all_aboard"), sum("passengers").alias("passengers"), sum("crew").alias("crew"),\
                                            sum("fatalities").alias("fatalities"), sum("passenger_fatalities").alias("passenger_fatalities"),\
                                            sum("crew_fatalities").alias("crew_fatalities"), sum("ground").alias("ground"))

In [18]:
aviaton_year.limit(3).pandas_api()

,year,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground
0,1970,2618.0,2208.0,368.0,1920.0,1570.0,283.0,9.0
1,1975,2167.0,1934.0,233.0,1460.0,1280.0,180.0,9.0
2,1977,9012923.0,2314.0,341.0,2098.0,1776.0,276.0,12.0


In [19]:
from pyspark.sql.functions import split

# Supongamos que tienes una columna llamada "columna_original" con valores separados por ","
aviation = aviation.withColumn("columna_dividida", split(aviation.location, ","))

# Obtén cada parte dividida en columnas separadas
aviation = aviation.withColumn("location_city", aviation.columna_dividida.getItem(0))
aviation = aviation.withColumn("location_country", aviation.columna_dividida.getItem(1))

In [20]:
aviation.limit(3).pandas_api()

,location,route,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,year,columna_dividida,location_city,location_country
0,"Stockholm, Sweden",Stockholm - Zurich,10,7,3,5,5,0,0,1970,"[Stockholm, Sweden]",Stockholm,Sweden
1,Near Villia Greece,Military exercise,27,22,5,23,19,4,0,1970,[Near Villia Greece],Near Villia Greece,None
2,"Faleolo, Western Samoa","Apia, Western Samoa - Pago Pago, American Samoa",32,29,3,32,29,3,0,1970,"[Faleolo, Western Samoa]",Faleolo,Western Samoa


In [21]:
aviation = aviation.drop("location","columna_dividida")

In [22]:
aviation.limit(3).pandas_api()

,route,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,year,location_city,location_country
0,Stockholm - Zurich,10,7,3,5,5,0,0,1970,Stockholm,Sweden
1,Military exercise,27,22,5,23,19,4,0,1970,Near Villia Greece,None
2,"Apia, Western Samoa - Pago Pago, American Samoa",32,29,3,32,29,3,0,1970,Faleolo,Western Samoa


In [23]:
column_order = ["year", "location_city", "location_country", "route", "all_aboard", "passengers", "crew", "fatalities", "passenger_fatalities" ,"crew_fatalities", "ground"]

# Seleccionar las columnas en el nuevo orden
aviation = aviation.select(column_order + [column for column in aviation.columns if column not in column_order])

In [24]:
aviation.limit(10).pandas_api()


,year,location_city,location_country,route,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground
0,1970,Stockholm,Sweden,Stockholm - Zurich,10,7,3,5,5,0,0
1,1970,Near Villia Greece,None,Military exercise,27,22,5,23,19,4,0
2,1970,Faleolo,Western Samoa,"Apia, Western Samoa - Pago Pago, American Samoa",32,29,3,32,29,3,0
3,1970,Mt. Pumacona,Peru,Trujillo - Juanjui,28,24,4,28,24,4,0
4,1970,Near Delhi,India,Kathmandu - Delhi,23,18,5,1,0,1,0
5,1970,Near Poza Rica,Mexico,Mexico City - Poza Rica,19,15,4,18,14,4,0
6,1970,Near Batagai,Russia,Chokurdakh - Batagay,34,28,6,34,28,6,0
7,1970,Cleveland,Ohio,Cleveland - Detroit,9,7,2,9,7,2,0
8,1970,Near Murmansk,Russia,Leningrad - Murmansk,11,32,6,11,8,3,0
9,1970,Apuseni mountains,Romania,Bucharest - Oradea,22,18,4,21,17,4,0


In [25]:
aviation.coalesce(1).write.csv("aviation_", header=True, mode="overwrite")

23/05/24 11:48:02 WARN TaskSetManager: Stage 18 contains a task of very large size (1852 KiB). The maximum recommended task size is 1000 KiB.


In [26]:
file = pd.read_csv("file.csv")

In [27]:

aviation.coalesce(1).write.csv("aviation", header = True, mode="overwrite")

23/05/24 11:48:40 WARN TaskSetManager: Stage 19 contains a task of very large size (1852 KiB). The maximum recommended task size is 1000 KiB.


Asumo, que ruta es lugar del sinistro. Cambiar el nombre

Dataset world bank

In [28]:
import requests
import zipfile
import io

In [29]:
# URL del archivo ZIP

In [30]:
url_flights = "https://api.worldbank.org/v2/es/indicator/IS.AIR.DPRT?downloadformat=csv"
url_passengers ="https://api.worldbank.org/v2/es/indicator/IS.AIR.PSGR?downloadformat=csv"

In [31]:
# Realizar la solicitud GET a la URL
response_flights = requests.get(url_flights)
response_passengers = requests.get(url_passengers)

Verificar si la solicitud fue exitosa

In [32]:
if response_flights.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_flights.content), "r") as zip_flights:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_flights.extractall()

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_flights.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [33]:
# Verificar si la solicitud fue exitosa
if response_passengers.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_passengers.content), "r") as zip_passengers:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_passengers.extractall()

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_passengers.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


Los códigos de estado HTTP son números de tres dígitos que se utilizan para indicar el resultado de una solicitud al servidor. Algunos ejemplos comunes de códigos de estado son:

    200 OK: La solicitud ha sido exitosa.
    400 Bad Request: La solicitud no se pudo entender o contiene sintaxis incorrecta.
    401 Unauthorized: La solicitud requiere autenticación o el usuario no tiene los permisos necesarios.
    404 Not Found: El recurso solicitado no se encuentra en el servidor.
    500 Internal Server Error: El servidor encontró una condición inesperada que le impidió cumplir con la solicitud.

In [34]:
import csv

# Ruta del archivo CSV
csv_flights = "API_IS.AIR.DPRT_DS2_es_csv_v2_5463649.csv"
csv_passengers = "API_IS.AIR.PSGR_DS2_es_csv_v2_5461838.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_flights = []
rows_passengers = []

In [35]:
# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_flights, "r") as file_flights:
    csv_reader_flights = csv.reader(file_flights)
    rows_flights = list(csv_reader_flights)
with open(csv_passengers, "r") as file_passengers:
    csv_reader_passengers = csv.reader(file_passengers)
    rows_passengers = list(csv_reader_passengers)

In [36]:
# Eliminar las primeras 4 filas
rows_flights = rows_flights[4:]
rows_passengers = rows_passengers[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset_flights.csv", "w", newline="") as file_flights:
    csv_writer_flights = csv.writer(file_flights)
    csv_writer_flights.writerows(rows_flights)
with open("dataset_passengers.csv", "w", newline="") as file_passengers:
    csv_writer_passengers = csv.writer(file_passengers)
    csv_writer_passengers.writerows(rows_passengers)

In [37]:
dataset_flights = pd.read_csv("dataset_flights.csv")
dataset_passengers = pd.read_csv("dataset_passengers.csv")

Transporte áreo, partidas de vuelos en todo el mundo de compañías registradas en el país
Algunos metadatos están solo en inglés

Las partidas de vuelos en todo el mundo de compañías registradas en el país son los despegues internos y despegues en el exterior de transportistas aéreos registrados en el país.

    ID: IS.AIR.DPRT
    Fuente: Organización de Aviación Civil Internacional ( OACI ), estadísticas mundiales de aviación civil y estimaciones de personal de la OACI.

In [38]:
dataset_flights =  spark.createDataFrame(dataset_flights)

In [39]:
print(dataset_flights.columns)

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Unnamed: 67']


In [40]:
dataset_flights

DataFrame[Country Name: string, Country Code: string, Indicator Name: string, Indicator Code: string, 1960: double, 1961: double, 1962: double, 1963: double, 1964: double, 1965: double, 1966: double, 1967: double, 1968: double, 1969: double, 1970: double, 1971: double, 1972: double, 1973: double, 1974: double, 1975: double, 1976: double, 1977: double, 1978: double, 1979: double, 1980: double, 1981: double, 1982: double, 1983: double, 1984: double, 1985: double, 1986: double, 1987: double, 1988: double, 1989: double, 1990: double, 1991: double, 1992: double, 1993: double, 1994: double, 1995: double, 1996: double, 1997: double, 1998: double, 1999: double, 2000: double, 2001: double, 2002: double, 2003: double, 2004: double, 2005: double, 2006: double, 2007: double, 2008: double, 2009: double, 2010: double, 2011: double, 2012: double, 2013: double, 2014: double, 2015: double, 2016: double, 2017: double, 2018: double, 2019: double, 2020: double, 2021: double, 2022: double, Unnamed: 67: dou

Obtener las columnas de tipo double

In [41]:
double_columns_flights = [col_name for col_name, col_type in dataset_flights.dtypes if col_type == "double"]

Cambiar el tipo de las columnas a entero

In [42]:
for column in double_columns_flights:
    dataset_flights = dataset_flights.withColumn(column, col(column).cast("int"))

In [43]:
dataset_flights

DataFrame[Country Name: string, Country Code: string, Indicator Name: string, Indicator Code: string, 1960: int, 1961: int, 1962: int, 1963: int, 1964: int, 1965: int, 1966: int, 1967: int, 1968: int, 1969: int, 1970: int, 1971: int, 1972: int, 1973: int, 1974: int, 1975: int, 1976: int, 1977: int, 1978: int, 1979: int, 1980: int, 1981: int, 1982: int, 1983: int, 1984: int, 1985: int, 1986: int, 1987: int, 1988: int, 1989: int, 1990: int, 1991: int, 1992: int, 1993: int, 1994: int, 1995: int, 1996: int, 1997: int, 1998: int, 1999: int, 2000: int, 2001: int, 2002: int, 2003: int, 2004: int, 2005: int, 2006: int, 2007: int, 2008: int, 2009: int, 2010: int, 2011: int, 2012: int, 2013: int, 2014: int, 2015: int, 2016: int, 2017: int, 2018: int, 2019: int, 2020: int, 2021: int, 2022: int, Unnamed: 67: int]

In [44]:
null_columns_dataset_fligths = [column for column in dataset_flights.columns if dataset_flights.select(column).dropna().count() == 0]

23/05/24 11:49:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [45]:
print(null_columns_dataset_fligths)

[]


Obtener las columnas con todos los valores nulos

In [46]:
columnas_nulas = [columna for columna in dataset_flights.columns if dataset_flights.where(col(columna).isNull()).count() == dataset_flights.count()]

In [47]:
null_columns = ["1960", "1961", "1962", "1963", "1964", "1965", "1966", "1967", "1968", "1969","Unnamed: 67"]

Eliminar las columnas con todos los valores nulos

In [48]:
dataset_flights = dataset_flights.drop(*null_columns)

In [49]:
dataset_flights.limit(3).pandas_api()

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"Transporte áreo, partidas de vuelos en todo el...",IS.AIR.DPRT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2132,2276,0,0,0,0
1,NaN,AFE,"Transporte áreo, partidas de vuelos en todo el...",IS.AIR.DPRT,152300,160400,157100,155800,150400,173500,193500,161800,170300,188200,204800,211800,206700,208000,207500,211900,195400,203000,202200,220100,232100,227200,220400,233800,233400,212500,231500,259600,253900,268400,282913,288922,267907,289028,300359,321298,319498,359120,318638,327145,470912,500629,514462,548834,534810,556341,562927,630147,705127,717795,286064,399895,0
2,Afganistán,AFG,"Transporte áreo, partidas de vuelos en todo el...",IS.AIR.DPRT,4000,4700,4800,3700,4300,4700,4500,4900,4800,4900,4200,4200,5400,5700,6200,6400,5800,6000,5900,5200,5300,5300,5300,5100,7000,7300,7500,4600,1900,3400,3409,0,0,0,0,0,0,0,0,0,21677,25021,17775,21696,25920,23532,22770,24207,10454,7334,4635,2865,0


In [50]:
dataset_flights.coalesce(1).write.csv("dataset_flights", header=True, mode="overwrite")

In [51]:
dataset_flights_total = dataset_flights.drop(col("Country Name"), col("Country Code"), col("Indicator Name"), col("Indicator Code"),col("Unnamed: 67"))

In [52]:
dataset_flights_total

DataFrame[1970: int, 1971: int, 1972: int, 1973: int, 1974: int, 1975: int, 1976: int, 1977: int, 1978: int, 1979: int, 1980: int, 1981: int, 1982: int, 1983: int, 1984: int, 1985: int, 1986: int, 1987: int, 1988: int, 1989: int, 1990: int, 1991: int, 1992: int, 1993: int, 1994: int, 1995: int, 1996: int, 1997: int, 1998: int, 1999: int, 2000: int, 2001: int, 2002: int, 2003: int, 2004: int, 2005: int, 2006: int, 2007: int, 2008: int, 2009: int, 2010: int, 2011: int, 2012: int, 2013: int, 2014: int, 2015: int, 2016: int, 2017: int, 2018: int, 2019: int, 2020: int, 2021: int, 2022: int]

In [53]:
dataset_flights_total.limit(3).pandas_api()

,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2132,2276,0,0,0,0
1,152300,160400,157100,155800,150400,173500,193500,161800,170300,188200,204800,211800,206700,208000,207500,211900,195400,203000,202200,220100,232100,227200,220400,233800,233400,212500,231500,259600,253900,268400,282913,288922,267907,289028,300359,321298,319498,359120,318638,327145,470912,500629,514462,548834,534810,556341,562927,630147,705127,717795,286064,399895,0
2,4000,4700,4800,3700,4300,4700,4500,4900,4800,4900,4200,4200,5400,5700,6200,6400,5800,6000,5900,5200,5300,5300,5300,5100,7000,7300,7500,4600,1900,3400,3409,0,0,0,0,0,0,0,0,0,21677,25021,17775,21696,25920,23532,22770,24207,10454,7334,4635,2865,0


In [54]:
dataset_flights_total = dataset_flights_total.na.fill(0)

In [55]:
dataset_flights_total.limit(10).pandas_api()

,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2132,2276,0,0,0,0
1,152300,160400,157100,155800,150400,173500,193500,161800,170300,188200,204800,211800,206700,208000,207500,211900,195400,203000,202200,220100,232100,227200,220400,233800,233400,212500,231500,259600,253900,268400,282913,288922,267907,289028,300359,321298,319498,359120,318638,327145,470912,500629,514462,548834,534810,556341,562927,630147,705127,717795,286064,399895,0
2,4000,4700,4800,3700,4300,4700,4500,4900,4800,4900,4200,4200,5400,5700,6200,6400,5800,6000,5900,5200,5300,5300,5300,5100,7000,7300,7500,4600,1900,3400,3409,0,0,0,0,0,0,0,0,0,21677,25021,17775,21696,25920,23532,22770,24207,10454,7334,4635,2865,0
3,67700,67600,67600,72400,69700,79700,88300,100800,105700,112800,116600,122800,116700,121300,119100,122300,109400,103600,98300,91400,85000,94700,81900,68900,70100,66400,62100,78000,78400,69400,71122,60104,52346,49482,56301,0,0,0,0,0,120773,138169,146632,155038,145526,164614,157788,151203,157126,158874,92611,134532,0
4,0,0,0,0,0,0,2600,4800,5000,11900,12800,15000,13900,22400,10700,13100,11100,11100,11000,6800,6900,7000,6900,5600,7600,7300,7700,7300,7400,5100,4399,4354,4343,4633,4849,4948,4965,5277,3361,3236,17060,12282,13072,14496,13716,13116,15482,13494,13978,13647,3792,3805,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,600,600,1400,1900,1100,2700,3885,3974,3762,3800,4104,4309,4481,4930,5058,5140,9412,10373,10309,11196,1992,0,306,1904,2935,2558,1274,1471,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,145300,151200,159800,173400,201400,215500,252000,274500,295700,314400,330000,341600,324500,352300,349000,344800,340700,334500,355500,356900,354400,319800,370600,390200,405800,388600,408400,407400,402500,441200,470300,470936,473898,493857,553147,431300,471714,516679,504211,811221,1065038,1093318,1159025,1222183,1289638,1375862,1476812,1503553,1539592,1599362,664879,920600,0
8,0,0,0,0,4700,5400,6200,6300,7200,8300,9500,10000,8700,9500,10100,10500,12200,12800,15100,16000,18800,19400,26300,30100,34000,34200,38000,39000,41100,44400,48330,50783,54952,71224,87217,0,0,0,0,170588,240855,269521,311758,327076,352224,437638,463947,459137,455956,426157,185260,234612,0
9,71700,75600,79000,71100,79000,89800,90000,93600,96500,103700,114400,113800,103900,109100,101400,100500,102300,111800,106200,101600,114000,78500,86300,83300,99500,112400,131400,171600,144600,183600,168815,124493,113178,91657,90518,81298,74162,79465,74649,74546,92847,90214,101716,145136,134521,145585,149334,146631,161862,163106,27447,54218,0


In [56]:
from pyspark.sql.functions import sum, col

In [57]:
# Calcular la suma acumulada de cada columna

In [58]:
sum_flights = dataset_flights_total.agg(*[sum(col(column)).alias(f"{column}") for column in dataset_flights_total.columns])

In [59]:
# Obtener los encabezados y los valores de la suma acumulada

In [60]:
headers_flights = sum_flights.columns
values_flights = sum_flights.first()

In [61]:
flights = spark.createDataFrame([(header_flights, value_flights) for header_flights, value_flights in zip(headers_flights, values_flights)], ["year", "flights"])

In [62]:
# Mostrar el nuevo DataFrame
flights.limit(3).pandas_api()

,year,flights
0,1970,57192900
1,1971,57492000
2,1972,48265300


Transporte aéreo, pasajeros transportados
Algunos metadatos están solo en inglés

Los pasajeros aéreos transportados incluyen tanto los pasajeros de las aeronaves domésticas como de las internacionales de las aerolíneas registradas en el país.

    ID: IS.AIR.PSGR
    Fuente: Organización de Aviación Civil Internacional ( OACI ), estadísticas mundiales de aviación civil y estimaciones de personal de la OACI

In [63]:
dataset_passengers =  spark.createDataFrame(dataset_passengers)

In [64]:
print(dataset_passengers.columns)

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', 'Unnamed: 67']


In [65]:
dataset_passengers

DataFrame[Country Name: string, Country Code: string, Indicator Name: string, Indicator Code: string, 1960: double, 1961: double, 1962: double, 1963: double, 1964: double, 1965: double, 1966: double, 1967: double, 1968: double, 1969: double, 1970: double, 1971: double, 1972: double, 1973: double, 1974: double, 1975: double, 1976: double, 1977: double, 1978: double, 1979: double, 1980: double, 1981: double, 1982: double, 1983: double, 1984: double, 1985: double, 1986: double, 1987: double, 1988: double, 1989: double, 1990: double, 1991: double, 1992: double, 1993: double, 1994: double, 1995: double, 1996: double, 1997: double, 1998: double, 1999: double, 2000: double, 2001: double, 2002: double, 2003: double, 2004: double, 2005: double, 2006: double, 2007: double, 2008: double, 2009: double, 2010: double, 2011: double, 2012: double, 2013: double, 2014: double, 2015: double, 2016: double, 2017: double, 2018: double, 2019: double, 2020: double, 2021: double, 2022: double, Unnamed: 67: dou

Obtener las columnas de tipo double

In [66]:
double_columns_passengers = [col_name for col_name, col_type in dataset_passengers.dtypes if col_type == "double"]

Cambiar el tipo de las columnas a entero

In [67]:
for column in double_columns_passengers:
    dataset_passengers = dataset_passengers.withColumn(column, col(column).cast("int"))

In [68]:
dataset_passengers

DataFrame[Country Name: string, Country Code: string, Indicator Name: string, Indicator Code: string, 1960: int, 1961: int, 1962: int, 1963: int, 1964: int, 1965: int, 1966: int, 1967: int, 1968: int, 1969: int, 1970: int, 1971: int, 1972: int, 1973: int, 1974: int, 1975: int, 1976: int, 1977: int, 1978: int, 1979: int, 1980: int, 1981: int, 1982: int, 1983: int, 1984: int, 1985: int, 1986: int, 1987: int, 1988: int, 1989: int, 1990: int, 1991: int, 1992: int, 1993: int, 1994: int, 1995: int, 1996: int, 1997: int, 1998: int, 1999: int, 2000: int, 2001: int, 2002: int, 2003: int, 2004: int, 2005: int, 2006: int, 2007: int, 2008: int, 2009: int, 2010: int, 2011: int, 2012: int, 2013: int, 2014: int, 2015: int, 2016: int, 2017: int, 2018: int, 2019: int, 2020: int, 2021: int, 2022: int, Unnamed: 67: int]

In [69]:
null_columns_dataset_passengers = [column for column in dataset_passengers.columns if dataset_passengers.select(column).dropna().count() == 0]

In [70]:
print(null_columns_dataset_passengers)

[]


Eliminar las columnas con todos los valores nulos

In [71]:
dataset_passengers = dataset_passengers.drop(*null_columns_dataset_passengers)

In [72]:
dataset_passengers.limit(3).pandas_api()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Transporte aéreo, pasajeros transportados",IS.AIR.PSGR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,223502,274280,0,0,0,0,0
1,NaN,AFE,"Transporte aéreo, pasajeros transportados",IS.AIR.PSGR,0,0,0,0,0,0,0,0,0,0,3418300,3775300,4074500,4808000,5126100,5333100,6040400,5798900,6195600,7515200,8614600,9063100,9088900,8973900,9474400,9385300,8825600,9837400,10931600,11110800,11257400,10802900,10302700,11082400,11775500,12153400,13178400,13456500,13234700,14038500,14963609,14546112,14462573,15802323,17562611,20661751,22039609,23295972,23988552,23800085,30374164,33507824,35838679,35112457,36350758,39154692,41553429,45423814,52613964,54851421,19166036,24057585,0,0
2,Afganistán,AFG,"Transporte aéreo, pasajeros transportados",IS.AIR.PSGR,0,0,0,0,0,0,0,0,0,0,84700,97400,104900,96600,96100,99000,101700,111200,100400,93000,76200,129100,181000,200200,220400,225500,184400,217900,201000,220000,241400,212300,212300,197000,238400,250400,255600,89600,52700,140200,149705,0,0,0,0,0,0,0,0,0,1999127,2279341,1737962,2044188,2209428,1929907,1917924,1647425,1125367,1066747,449041,293212,0,0


In [73]:
dataset_passengers_total = dataset_passengers.drop(col("Country Name"), col("Country Code"), col("Indicator Name"), col("Indicator Code"),col("Unnamed: 67"))

In [74]:
dataset_passengers_total.limit(3).pandas_api()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,223502,274280,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,3418300,3775300,4074500,4808000,5126100,5333100,6040400,5798900,6195600,7515200,8614600,9063100,9088900,8973900,9474400,9385300,8825600,9837400,10931600,11110800,11257400,10802900,10302700,11082400,11775500,12153400,13178400,13456500,13234700,14038500,14963609,14546112,14462573,15802323,17562611,20661751,22039609,23295972,23988552,23800085,30374164,33507824,35838679,35112457,36350758,39154692,41553429,45423814,52613964,54851421,19166036,24057585,0
2,0,0,0,0,0,0,0,0,0,0,84700,97400,104900,96600,96100,99000,101700,111200,100400,93000,76200,129100,181000,200200,220400,225500,184400,217900,201000,220000,241400,212300,212300,197000,238400,250400,255600,89600,52700,140200,149705,0,0,0,0,0,0,0,0,0,1999127,2279341,1737962,2044188,2209428,1929907,1917924,1647425,1125367,1066747,449041,293212,0


In [75]:
dataset_passengers_total = dataset_passengers_total.na.fill(0)

In [76]:
dataset_passengers_total.limit(3).pandas_api()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,223502,274280,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,3418300,3775300,4074500,4808000,5126100,5333100,6040400,5798900,6195600,7515200,8614600,9063100,9088900,8973900,9474400,9385300,8825600,9837400,10931600,11110800,11257400,10802900,10302700,11082400,11775500,12153400,13178400,13456500,13234700,14038500,14963609,14546112,14462573,15802323,17562611,20661751,22039609,23295972,23988552,23800085,30374164,33507824,35838679,35112457,36350758,39154692,41553429,45423814,52613964,54851421,19166036,24057585,0
2,0,0,0,0,0,0,0,0,0,0,84700,97400,104900,96600,96100,99000,101700,111200,100400,93000,76200,129100,181000,200200,220400,225500,184400,217900,201000,220000,241400,212300,212300,197000,238400,250400,255600,89600,52700,140200,149705,0,0,0,0,0,0,0,0,0,1999127,2279341,1737962,2044188,2209428,1929907,1917924,1647425,1125367,1066747,449041,293212,0


Calcular la suma acumulada de cada columna

In [77]:
sum_passengers = dataset_passengers_total.agg(*[sum(col(column)).alias(f"{column}") for column in dataset_passengers_total.columns])

Obtener los encabezados y los valores de la suma acumulada

In [78]:
headers_passengers = sum_passengers.columns
values_passengers = sum_passengers.first()

In [79]:
passengers = spark.createDataFrame([(header_passengers, value_passengers) for header_passengers, value_passengers in zip(headers_passengers, values_passengers)], ["year", "total_passengers"])

Mostrar el nuevo DataFrame

In [80]:
passengers.limit(3).pandas_api()

,year,total_passengers
0,1960,0
1,1961,0
2,1962,0


In [81]:
from pyspark.sql.functions import count
number_accidents = aviation.groupBy("year").agg(count("*").alias("num_accidents")).orderBy("year")

In [82]:
number_accidents.limit(3).pandas_api()

,year,num_accidents
0,1970,73
1,1971,58
2,1972,77


In [83]:
dataframe_final = flights.join(passengers, "year").join(aviaton_year, "year").join(number_accidents, "year")

In [84]:
dataframe_final.orderBy("year").limit(5).pandas_api()

,year,flights,total_passengers,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,num_accidents
0,1970,57192900,1861543452,2618.0,2208.0,368.0,1920.0,1570.0,283.0,9.0,73
1,1971,57492000,1991897624,5346018.0,2134.0,300.0,1772.0,1563.0,232.0,2.0,58
2,1972,48265300,1830207940,9446147.0,2971.0,484.0,2667.0,2433.0,423.0,65.0,77
3,1973,59114800,2403325100,189005050.0,2942.0,437.0,2252.0,2040.0,339.0,24.0,64
4,1974,64818500,2871103800,2878.0,2592.0,286.0,2242.0,2213.0,221.0,0.0,58


In [85]:
columns_dataframe_final = dataframe_final.columns

Presenta inconsistencia la columna `all_board`, eliminar y generararla `all_board`= `passengers` + `crew`

In [86]:
dataframe_final = dataframe_final.drop(col("all_aboard"))

In [87]:
dataframe_final.limit(3).toPandas()

,year,flights,total_passengers,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,num_accidents
0,1970,57192900,1861543452,2208.0,368.0,1920.0,1570.0,283.0,9.0,73
1,1975,65489300,2985818000,1934.0,233.0,1460.0,1280.0,180.0,9.0,51
2,1977,69969700,3564133252,2314.0,341.0,2098.0,1776.0,276.0,12.0,63


In [88]:
dataframe_final = dataframe_final.withColumn("all_aboard", col("passengers") + col("crew"))

In [89]:
dataframe_final = dataframe_final.select(*columns_dataframe_final)

In [90]:
dataframe_final.limit(3).pandas_api()

,year,flights,total_passengers,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,num_accidents
0,1970,57192900,1861543452,2576.0,2208.0,368.0,1920.0,1570.0,283.0,9.0,73
1,1975,65489300,2985818000,2167.0,1934.0,233.0,1460.0,1280.0,180.0,9.0,51
2,1977,69969700,3564133252,2655.0,2314.0,341.0,2098.0,1776.0,276.0,12.0,63


In [91]:
dataframe_final = dataframe_final.withColumn("ratio_passengers", (col("fatalities") / col("total_passengers"))*1000000)
dataframe_final = dataframe_final.withColumn("ratio_fligts", (col("num_accidents") / col("flights"))*1000000)
dataframe_final = dataframe_final.withColumn("dif_ratio", (col("ratio_passengers") - col("ratio_fligts")))

In [92]:
dataframe_final = dataframe_final.orderBy("year")

In [93]:
dataframe_final.limit(3).pandas_api()

,year,flights,total_passengers,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,num_accidents,ratio_passengers,ratio_fligts,dif_ratio
0,1970,57192900,1861543452,2576.0,2208.0,368.0,1920.0,1570.0,283.0,9.0,73,1.031402,1.276382,-0.244980
1,1971,57492000,1991897624,2434.0,2134.0,300.0,1772.0,1563.0,232.0,2.0,58,0.889604,1.008836,-0.119232
2,1972,48265300,1830207940,3455.0,2971.0,484.0,2667.0,2433.0,423.0,65.0,77,1.457211,1.595349,-0.138138


In [94]:
from pyspark.sql import functions as F

Redondear el valor de la columna 'year' al múltiplo más cercano de 5

In [95]:
dataframe_final = dataframe_final.withColumn("year_group", F.floor(dataframe_final["year"] / 5) * 5)

In [96]:
# Crear una nueva columna que represente el intervalo de 5 años

In [97]:
dataframe_final = dataframe_final.withColumn("year_interval", F.concat(dataframe_final["year_group"], F.lit("-"), dataframe_final["year_group"] + 4))

In [98]:
# Mostrar el DataFrame resultante

In [99]:
dataframe_final.limit(6).pandas_api()

,year,flights,total_passengers,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,num_accidents,ratio_passengers,ratio_fligts,dif_ratio,year_group,year_interval
0,1970,57192900,1861543452,2576.0,2208.0,368.0,1920.0,1570.0,283.0,9.0,73,1.031402,1.276382,-0.244980,1970,1970-1974
1,1971,57492000,1991897624,2434.0,2134.0,300.0,1772.0,1563.0,232.0,2.0,58,0.889604,1.008836,-0.119232,1970,1970-1974
2,1972,48265300,1830207940,3455.0,2971.0,484.0,2667.0,2433.0,423.0,65.0,77,1.457211,1.595349,-0.138138,1970,1970-1974
3,1973,59114800,2403325100,3379.0,2942.0,437.0,2252.0,2040.0,339.0,24.0,64,0.937035,1.082639,-0.145604,1970,1970-1974
4,1974,64818500,2871103800,2878.0,2592.0,286.0,2242.0,2213.0,221.0,0.0,58,0.780884,0.894806,-0.113922,1970,1970-1974
5,1975,65489300,2985818000,2167.0,1934.0,233.0,1460.0,1280.0,180.0,9.0,51,0.488978,0.778753,-0.289775,1975,1975-1979


In [100]:
dataframe_final.columns

['year',
 'flights',
 'total_passengers',
 'all_aboard',
 'passengers',
 'crew',
 'fatalities',
 'passenger_fatalities',
 'crew_fatalities',
 'ground',
 'num_accidents',
 'ratio_passengers',
 'ratio_fligts',
 'dif_ratio',
 'year_group',
 'year_interval']

In [101]:
dataframe_5year = dataframe_final.groupBy("year_interval").agg(sum('flights').alias('flights'), sum('total_passengers').alias('total_passengers'), \
sum('all_aboard').alias("all_aboard"), sum("passengers").alias("passengers"), sum("crew").alias("crew"), sum("fatalities").alias("fatalities"),\
sum("passenger_fatalities").alias("passenger_fatalities"), sum("crew_fatalities").alias("crew_fatalities"), sum("ground").alias("ground"),\
sum('num_accidents').alias("num_accidents"))

In [102]:
dataframe_5year = dataframe_5year.orderBy("year_interval")

In [103]:
dataframe_5year = dataframe_5year.withColumn("ratio_passengers", (col("fatalities") / col("total_passengers"))*1000000)
dataframe_5year = dataframe_5year.withColumn("ratio_fligts", (col("num_accidents") / col("flights"))*1000000)
dataframe_5year = dataframe_5year.withColumn("dif_ratio", (col("ratio_passengers") - col("ratio_fligts")))

In [104]:
dataframe_5year.limit(11).pandas_api()

,year_interval,flights,total_passengers,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground,num_accidents,ratio_passengers,ratio_fligts,dif_ratio
0,1970-1974,286883500,10958077916,14722.0,12847.0,1875.0,10853.0,9819.0,1498.0,100.0,330,0.990411,1.150293,-0.159882
1,1975-1979,350896100,18311457052,11981.0,10467.0,1514.0,8581.0,7437.0,1218.0,240.0,282,0.468614,0.803657,-0.335043
2,1980-1984,387035000,23689676780,10857.0,9603.0,1254.0,6988.0,5948.0,901.0,191.0,245,0.294981,0.633018,-0.338037
3,1985-1989,471513100,31241755296,13828.0,12285.0,1543.0,9715.0,8275.0,1130.0,218.0,308,0.310962,0.653216,-0.342254
4,1990-1994,570175900,41762375092,12370.0,10815.0,1555.0,8157.0,6688.0,1044.0,139.0,326,0.195319,0.571753,-0.376434
5,1995-1999,731657900,52935944572,11957.0,10516.0,1441.0,7808.0,6462.0,1107.0,406.0,305,0.147499,0.416861,-0.269362
6,2000-2004,828375937,63282512869,8234.0,7058.0,1176.0,6078.0,4976.0,938.0,5857.0,284,0.096045,0.342840,-0.246794
7,2005-2009,976071533,82601826858,8004.0,7086.0,918.0,5096.0,4341.0,696.0,168.0,223,0.061694,0.228467,-0.166773
8,2010-2014,1276804468,117419379398,4977.0,4350.0,627.0,3780.0,3097.0,522.0,94.0,150,0.032192,0.117481,-0.085288
9,2015-2019,1514525475,157915624755,3615.0,3217.0,398.0,2667.0,2343.0,324.0,80.0,88,0.016889,0.058104,-0.041215


In [105]:
dataframe_5year.write.csv("dataframe_5years", header=True, mode="overwrite")


In [106]:
data = pd.read_csv("dataframe_accidents.csv")

In [107]:
aviation.limit(9).pandas_api()

,year,location_city,location_country,route,all_aboard,passengers,crew,fatalities,passenger_fatalities,crew_fatalities,ground
0,1970,Stockholm,Sweden,Stockholm - Zurich,10,7,3,5,5,0,0
1,1970,Near Villia Greece,None,Military exercise,27,22,5,23,19,4,0
2,1970,Faleolo,Western Samoa,"Apia, Western Samoa - Pago Pago, American Samoa",32,29,3,32,29,3,0
3,1970,Mt. Pumacona,Peru,Trujillo - Juanjui,28,24,4,28,24,4,0
4,1970,Near Delhi,India,Kathmandu - Delhi,23,18,5,1,0,1,0
5,1970,Near Poza Rica,Mexico,Mexico City - Poza Rica,19,15,4,18,14,4,0
6,1970,Near Batagai,Russia,Chokurdakh - Batagay,34,28,6,34,28,6,0
7,1970,Cleveland,Ohio,Cleveland - Detroit,9,7,2,9,7,2,0
8,1970,Near Murmansk,Russia,Leningrad - Murmansk,11,32,6,11,8,3,0


In [108]:
mapa = aviation.groupBy('location_country').agg(sum('fatalities').alias('fatalities'), count('*').alias('accidents'))

In [109]:
mapa.limit(9).pandas_api()

,location_country,fatalities,accidents
0,Western Samoa,32.0,1
1,Dominican Republic,140.0,4
2,Colombia,1710.0,93
3,South Vietnam,925.0,20
4,Taiwan,885.0,25
5,US Virgin Islands,23.0,1
6,Ohio,31.0,11
7,Arizona,88.0,11
8,Japan,318.0,13


In [110]:
mapa_= pd.read_csv("mapa.csv")

In [111]:
mapa_

,location_country,fatalities,accidents
0,Western Samoa,32.0,1
1,Dominican Republic,140.0,4
2,Colombia,1710.0,93
3,South Vietnam,925.0,20
4,Taiwan,885.0,25
...,...,...,...
427,Congo Democratic Republic,19.0,1
428,USA,16.0,1
429,Nambia,33.0,1
430,Kauai,1.0,1


In [112]:
state_dict = {
    'Alabama': 'USA',
    'Alaska': 'USA',
    'Arizona': 'USA',
    'Arkansas': 'USA',
    'California': 'USA',
    'Colorado': 'USA',
    'Connecticut': 'USA',
    'Delaware': 'USA',
    'Florida': 'USA',
    'Georgia': 'USA',
    'Hawaii': 'USA',
    'Idaho': 'USA',
    'Illinois': 'USA',
    'Indiana': 'USA',
    'Iowa': 'USA',
    'Kansas': 'USA',
    'Kentucky': 'USA',
    'Louisiana': 'USA',
    'Maine': 'USA',
    'Maryland': 'USA',
    'Massachusetts': 'USA',
    'Michigan': 'USA',
    'Minnesota': 'USA',
    'Mississippi': 'USA',
    'Missouri': 'USA',
    'Montana': 'USA',
    'Nebraska': 'USA',
    'Nevada': 'USA',
    'New Hampshire': 'USA',
    'New Jersey': 'USA',
    'New Mexico': 'USA',
    'New York': 'USA',
    'North Carolina': 'USA',
    'North Dakota': 'USA',
    'Ohio': 'USA',
    'Oklahoma': 'USA',
    'Oregon': 'USA',
    'Pennsylvania': 'USA',
    'Rhode Island': 'USA',
    'South Carolina': 'USA',
    'South Dakota': 'USA',
    'Tennessee': 'USA',
    'Texas': 'USA',
    'Utah': 'USA',
    'Vermont': 'USA',
    'Virginia': 'USA',
    'Washington': 'USA',
    'West Virginia': 'USA',
    'Wisconsin': 'USA',
    'Wyoming': 'USA'
}


In [113]:
mapa_['location_country'] = mapa_['location_country'].replace(state_dict)
mapa_['location_country'] = mapa_['location_country'].replace(state_dict)

mapa_.head(30)

,location_country,fatalities,accidents
0,Western Samoa,32.0,1
1,Dominican Republic,140.0,4
2,Colombia,1710.0,93
3,South Vietnam,925.0,20
4,Taiwan,885.0,25
5,US Virgin Islands,23.0,1
6,US Virgin Islands,31.0,11
7,US Virgin Islands,88.0,11
8,Japan,318.0,13
9,NaN,1328.0,57


In [114]:
mapa_.head(50)

,location_country,fatalities,accidents
0,Western Samoa,32.0,1
1,Dominican Republic,140.0,4
2,Colombia,1710.0,93
3,South Vietnam,925.0,20
4,Taiwan,885.0,25
5,US Virgin Islands,23.0,1
6,US Virgin Islands,31.0,11
7,US Virgin Islands,88.0,11
8,Japan,318.0,13
9,NaN,1328.0,57


In [115]:
mapa_.to_csv('mapa_.csv', index=False)